In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_anthropic import ChatAnthropic

# model = ChatAnthropic(model='claude-3-opus-20240229')

In [3]:
from dotenv import load_dotenv
load_dotenv()
KEY = os.getenv("ANTHROPIC_API_KEY")

In [4]:
llm = ChatAnthropic(api_key=KEY,model="claude-3-sonnet-20240229", temperature=0.2, max_tokens=1024)

In [5]:
RESPONSE_JSON = {
    "1": {
        "question": "Multiple choice question here",
        "options": {
            "A": "Option here",
            "B": "Option here",
            "C": "Option here",
            "D": "Option here"
        },
        "correct_option": "A"
    },
    "2": {
        "question": "Multiple choice question here",
        "options": {
            "A": "Option here",
            "B": "Option here",
            "C": "Option here",
            "D": "Option here"
        },
        "correct_option": "B"
    },
    "3": {
        "question": "Multiple choice question here",
        "options": {
            "A": "Option here",
            "B": "Option here",
            "C": "Option here",
            "D": "Option here"
        },
        "correct_option": "A"
    }
}


In [6]:
template ="""
    Text :{text}
    You are an expert MCQ maker. Given the above text, it is your job to 
    create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
    Make sure the questions are not repeated and check all the questions to be conforming the text
    as well. Make sure to format your response like RESPONSE_JSON below and use it as a guide.
    Ensure to make {number} MCQs
    ### RESPONSE_JSON
    {response_json}
"""

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
import PyPDF2


In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","tone","response_josn"],
    template=template
)

In [9]:
quiz_chain = LLMChain(llm=llm,prompt= quiz_generation_prompt,output_key="quiz")

C:\Users\HP\AppData\Local\Temp\ipykernel_11964\3118858387.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm,prompt= quiz_generation_prompt,output_key="quiz")


In [10]:
template2 = """ 
    You are an expert english grammarian and writer. GIven a Multiple Choice Quiz for {subject} students.
    You need to evaluate the complexity of the question and gibe a complete analysis of the quiz. Only use at max 50 words for complexity.
    if the quiz is not at per with the cognitive and analytical abilities of the students,
    update the quiz Questions which needs to be changed and change the tone such that it perfectly fits the students abilities.
    Quiz_MCQs:
    {quiz}
    Check from an expert English Writer of the above quiz
"""

In [11]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=template2
)

In [12]:
review_chain = LLMChain(llm=llm,prompt = quiz_evaluation_prompt,output_key="review")

In [13]:
generate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                                 output_variables=["quiz","review"],verbose = True)

In [14]:
with open(r'C:\Users\HP\Desktop\Genai_p1\data.txt','r') as file:
    TEXT = file.read()

print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]

From a theoretical 

In [15]:
json.dumps(RESPONSE_JSON)

'{"1": {"question": "Multiple choice question here", "options": {"A": "Option here", "B": "Option here", "C": "Option here", "D": "Option here"}, "correct_option": "A"}, "2": {"question": "Multiple choice question here", "options": {"A": "Option here", "B": "Option here", "C": "Option here", "D": "Option here"}, "correct_option": "B"}, "3": {"question": "Multiple choice question here", "options": {"A": "Option here", "B": "Option here", "C": "Option here", "D": "Option here"}, "correct_option": "A"}}'

In [34]:
NUM =5
SUBJECT="Machine Learning"
TONE = "simple"

In [35]:
response = generate_chain(
    {
        "text":TEXT,
        "number":NUM,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...

> Finished chain.


In [36]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFr

In [41]:
quiz = response.get("quiz")
quiz_cleaned = quiz.strip().strip('RESPONSE_JSON').strip('```')
print(quiz_cleaned)



{
  "1": {
    "question": "What is the primary focus of machine learning?",
    "options": {
      "A": "The development and study of statistical algorithms that can learn from data and generalize to unseen data",
      "B": "The creation of artificial intelligence systems",
      "C": "The study of human cognitive processes",
      "D": "The development of computer hardware"
    },
    "correct_option": "A"
  },
  "2": {
    "question": "Which field has recently surpassed many previous approaches in machine learning performance?",
    "options": {
      "A": "Computer vision",
      "B": "Natural language processing",
      "C": "Artificial neural networks",
      "D": "Speech recognition"
    },
    "correct_option": "C"
  },
  "3": {
    "question": "What is the name given to machine learning when applied to business problems?",
    "options": {
      "A": "Data mining",
      "B": "Predictive analytics",
      "C": "Exploratory data analysis",
      "D": "Computational statistics

In [42]:
quiz = json.loads(quiz_cleaned)

In [23]:
quiz_table_data = []
for key,value in quiz.items():
    mcq = value["question"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option,option_value in value["options"].items() 
        ]
    )
    correct = value["correct_option"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct}) 

In [24]:
quiz_table_data

[{'MCQ': 'What is Machine Learning (ML) primarily concerned with?',
  'Choices': 'A: The development and study of statistical algorithms that can learn from data and generalize to unseen data | B: The development of artificial intelligence systems | C: The study of human cognitive processes | D: The development of computer hardware',
  'Correct': 'A'},
 {'MCQ': 'Which of the following fields has Machine Learning found application in?',
  'Choices': 'A: Natural language processing | B: Computer vision | C: Speech recognition | D: All of the above',
  'Correct': 'D'},
 {'MCQ': 'What is the mathematical foundation of Machine Learning provided by?',
  'Choices': 'A: Mathematical optimization (mathematical programming) methods | B: Calculus | C: Linear algebra | D: Statistics',
  'Correct': 'A'},
 {'MCQ': "Who coined the term 'Machine Learning' in 1959?",
  'Choices': 'A: Donald Hebb | B: Arthur Samuel | C: Tom M. Mitchell | D: Alan Turing',
  'Correct': 'B'},
 {'MCQ': 'What are the two mai

In [43]:
quiz = pd.DataFrame(quiz_table_data)

In [46]:
fileName = SUBJECT+'.csv'
quiz.to_csv(f'{fileName}',index=False)

In [48]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'09_09_2024_19_18_57'